#### 코드 사용법 - 1. 넘파이

In [2]:
import numpy as np
print('파이썬 리스트를 사용한 연산:')
a = [1,2,3]
b = [4,5,6]
print('a+b:', a+b)
try:
  print(a*b)
except TypeError:
  print('a*b 파이썬 리스트에 대해 a*b와 같은 연산을 할 수 없음.')
print()
print('넘파이 배열을 사용한 연산:')
a = np.array([1,2,3])
b = np.array([4,5,6])
print('a+b:', a+b)
print('a*b:', a*b)

파이썬 리스트를 사용한 연산:
a+b: [1, 2, 3, 4, 5, 6]
a*b 파이썬 리스트에 대해 a*b와 같은 연산을 할 수 없음.

넘파이 배열을 사용한 연산:
a+b: [5 7 9]
a*b: [ 4 10 18]


In [3]:
a = np.array([[1,2],[3,4]])
print('a:')
print(a)
print('a.sum(axis=0):', a.sum(axis=0))
print('a.sum(axis=1):', a.sum(axis=1))

a:
[[1 2]
 [3 4]]
a.sum(axis=0): [4 6]
a.sum(axis=1): [3 7]


In [4]:
a = np.array([[1,2,3],[4,5,6]])

b = np.array([10,20,30])
print('a+b: \n',a+b)

a+b: 
 [[11 22 33]
 [14 25 36]]


#### 코드 사용법 -2. 타입을 확인하는 함수

In [5]:
from numpy import ndarray
def square(x: ndarray) -> ndarray:
  """
  인자로 받은 ndarray 배열의 각 요솟값을 제곱한다.
  """
  return np.power(x,2)

def leaky_relu(x: ndarray) -> ndarray:
  """
  ndarray 배열의 각 요소에 'Leaky ReLU'함수를 적용한다.
  """
  return np.maximum(0.2 * x, x)

#### 도함수

In [6]:
from typing import Callable

def deriv(func: Callable[[ndarray], ndarray],
          input_: ndarray,
          delta: float = 0.001) -> ndarray:
          """
          배열 input의 각 요소에 대해 함수 func의 도함숫값 계산
          """
          return (func(input_ + delta) - func(input_ - delta)) / (2 * delta)

#### 합성함수

In [7]:
from typing import List

# ndarray를 인자로 받고 ndarray를 반환하는 함수
Array_Function = Callable[[ndarray], ndarray]

# Chain은 함수의 리스트다.
Chain = List[Array_Function]

def chain_length_2(chain: Chain,
                   a: ndarray) -> ndarray :
  """
  두 함수를 연쇄(chain)적으로 평가
  """
  assert len(chain) == 2, "인자 chain의 길이는 2여야 함"

  f1 = chain[0]
  f2 = chain[1]
  
  return f2(f1(x))

#### 연쇄법칙

In [8]:
def sigmoid(x: ndarray) -> ndarray:
  """
  입력으로 받은 ndarray의 각 요소에 대한 sigmoid 함숫값을 계산한다.
  """
  return 1 / (1 + np.exp(-x))

In [9]:
def chain_deriv_2(chain: Chain,
                  input_range: ndarray) -> ndarray:
  """
  두 함수로 구성된 합성함수의 도함수를 계산하기 위해 연쇄법칙을 사용함
  (f2(f1(x)))' = f2'(f1(x)) * f1'(x)
  """

  assert len(chain) == 2, "인자 chain의 길이는 2여야 함"

  assert input_range.ndim == 1, "input_range는 1차원 ndarray여야 함"

  f1 = chain[0]
  f2 = chain[1]

  # df1/dx
  f1_of_x = f1(input_range)

  # df1/du
  df1dx = deriv(f1, input_range)

  # df2/du(f1(x))
  df2du = deriv(f2, f1(input_range))

  # 각 점끼리 값을 곱함
  return df1dx * df2du

#### 조금 더 복잡한 예제

In [10]:
# 합성함수를 구성하는 함수를 알고 있을 때, 그 합성함수의 도함수를 구하는 수식을 코드로 구현
def chain_deriv_3(chain: Chain,
                  ipnut_range: ndarray) -> ndarray:
  """
  세 함수로 구성된 합성함수의 도함수를 계산하기 위해 연쇄법칙을 사용함
  (f3(f2(f1)))' = f3'(f2(f1(X))) * f2'(f1(x)) * f1(x)
  """

  assert len(chain) == 3, '인자 chain의 길이는 3이어야 함'

  f1 = chain[0]
  f2 = chain[1]
  f3 = chain[2]

  # f1(x)
  f1_of_x = f1(input_range)

  # f2(f1(x))
  f2_of_x = f2(f1_of_x)

  # df3du
  df3du = deriv(f3, f2_of_x)

  # df2du
  df2du = deriv(f2, f1_of_x)

  # df1dx
  df1dx = deriv(f1, input_range)

  # 각 점끼리 값을 곱함
  return df1dx * df2du * df3du

#### 입력이 두 개 이상인 함수의 합성함수

In [11]:
def multiple_inputs_add(x: ndarray,
                        y: ndarray,
                        sigma: Array_Function) -> float:
  """
  두 개의 입력을 받아 값을 더하는 함수의 순방향 계산
  """
  assert x.shape == y.shape

  a = x + y
  return sigma(a)

#### 입력이 여러 개인 함수의 도함수

In [12]:
def multiple_inputs_add_backward(x: ndarray,
                                 y: ndarray,
                                 sigma: Array_Function) -> float:
  """
  두 개의 입력을 받는 함수의 두 입력에 대한 각각의 도함수 계산
  """
  # 정방향 계산 수행
  a = x + y

  # 도함수 계산
  dsda = deriv(sigma, a)

  dadx, dady = 1, 1

  return dsda * dadx, dsda * dady